In [3]:
!pip install --upgrade plotly-geo
!pip install --upgrade geopandas
!pip install --upgrade pyshp
!pip install --upgrade shapely

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import plotly.figure_factory as ff
import plotly.express as px
import time
from datetime import datetime
import warnings
warnings.filterwarnings("ignore")

datafile = pd.read_csv(r"C:\Users\HPC500C602\Downloads\president_state.csv")
state_codes = pd.read_csv(r"C:\Users\HPC500C602\Downloads\world_country_and_usa_states_latitude_and_longitude_values.xls")
datafile = datafile.merge(state_codes, left_on='state', right_on='usa_state')
fig = px.choropleth(datafile, locations='usa_state_code', color="total_votes",
                           range_color=(0, 10000000),
                           locationmode = 'USA-states',  
                           scope="usa",
                           title='USA Presidential Votes Counts' 
                          )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

df_president_county = pd.read_csv(r"C:\Users\HPC500C602\Downloads\president_county_candidate_.csv")
df_president_county = df_president_county[df_president_county.party.isin(['DEM','REP'])]
df_president_county = df_president_county.groupby(['state','party'])["total_votes"].sum()
df_president_county = df_president_county.reset_index()
state_codes = pd.read_csv(r"C:\Users\HPC500C602\Downloads\world_country_and_usa_states_latitude_and_longitude_values.xls")
state_codes = state_codes[['usa_state','usa_state_code']]
df_president_county = df_president_county.merge(state_codes, left_on='state', right_on='usa_state')
df_president_county = df_president_county.drop(['usa_state'], axis=1)
df_president_county_dummy = pd.get_dummies(df_president_county['party'])
df_president_county = df_president_county.join(df_president_county_dummy)
df_president_county['DEM_votes'] = df_president_county['DEM'] * df_president_county['total_votes'] 
df_president_county['REP_votes'] = df_president_county['REP'] * df_president_county['total_votes'] 
df_president_county = df_president_county.groupby(['state','usa_state_code'])["DEM_votes","REP_votes"].sum()
df_president_county = df_president_county.reset_index()
df_president_county['percent_vote'] = df_president_county['DEM_votes']*100/(df_president_county['REP_votes']+df_president_county['DEM_votes'])
fig = px.choropleth(df_president_county, 
                    locations="usa_state_code", 
                    color = "percent_vote",
                    locationmode = 'USA-states', 
                    hover_name="state",
                    range_color=[25,75],
                    color_continuous_scale = 'RdBu',#blues
                    scope="usa",
                    title='2020 USA Election: Winning Party By State')
fig.show()